<a href="https://colab.research.google.com/github/SridharK12/Deep-Learning-Project/blob/main/Quora_Spam_Filter_Trained_embeddings_Submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(tf.__version__)
from google.colab import drive
drive.mount('/content/drive')  

2.8.0
Mounted at /content/drive


In [ ]:
vocab_size = 10000     #Number of unique words in vocabulary
embedding_dim = 64     #Dimensionality of embedding
  #maximum length of the data record. Only first 200 characters will be considered by the ealgorithms
trunc_type = 'post'    #When the truncation type is post characters to the end of text are truncated whenever the length reaches max length
padding_type = 'post'  #When the truncation type is pre initial characters are deleted when length of text exceeds max length 
oov_tok = '<OOV>'      #This is out of vocabulary token
training_portion = .8

In [ ]:
data = pd.read_csv("/content/drive/My Drive/quora_train.csv",encoding='iso-8859-1')

In [ ]:
data.shape

(1306122, 3)

In [ ]:
data_sample=data.sample(frac=0.05)

In [ ]:
data_sample.shape

(65306, 3)

In [ ]:
data_sample["target"].value_counts()

0    61278
1     4028
Name: target, dtype: int64

In [ ]:
4028/61278

0.06573321583602598

In [ ]:
data["target"].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [ ]:
80810/12253122

0.006595053897284301

In [ ]:
data_bkp=data
data=data_sample
#30 percent of the data is used for the algorithm to reduce the run time

The data is sampled for 5% and it got a better ratio of positive and negative.

This step is taken to reduce run time

In [ ]:
data['text_length']=data['question_text'].str.len()

In [ ]:
data.head()

,qid,question_text,target,text_length
65630,0cdedf58609ed5890c93,Why do most New Yorkers love to turn EVERY poi...,1,96
41050,0807742c47ca1c92551d,Can a new video rank above a video which has m...,0,94
1173916,e60e4992fb82cd57e23e,Why is finding an IT career path difficult?,0,43
1279403,fabe9bc6e45b579e89bb,What is the order with Wonder Woman and Batman...,0,58
697187,888785ee46ab5f33da33,Which bike is much better in b/w Suzuki Gixxer...,0,77


In [ ]:
data['text_length'].max()

474

In [ ]:
data['text_length'].mean()

70.66776406455762

In [ ]:
data['text_length'].median()

60.0

In [ ]:
data.shape

(65306, 4)

In [ ]:
data.describe()

,target,text_length
count,65306.000000,65306.000000
mean,0.061679,70.667764
std,0.240573,39.002972
min,0.000000,11.000000
25%,0.000000,45.000000
50%,0.000000,60.000000
75%,0.000000,85.000000
max,1.000000,474.000000


In [ ]:
max_length = 64    #max_length can be cosidered as 75% i 85

Now lets us do the text cleaning

In [ ]:
!pip install pyspellchecker
from spellchecker import SpellChecker
import re
import string
from nltk.corpus import stopwords

spell = SpellChecker()
def correct_spellings(x, spell=spell):
    """correct the missplled words of a given tweet"""
    x = x.split()
    misspelled = spell.unknown(x)
    result = map(lambda word : spell.correction(word) if word in  misspelled else word, x)
    return " ".join(result)

     |████████████████████████████████| 2.7 MB 12.7 MB/s 


In [ ]:
def questions_cleaning(x, correct_spelling=True, remove_emojis=True, remove_stop_words=True):
    """Apply function to a clean a tweet"""
    x = x.lower().strip()
    # romove urls
    url = re.compile(r'https?://\S+|www\.\S+')
    x = url.sub(r'',x)
    # remove html tags
    html = re.compile(r'<.*?>')
    x = html.sub(r'',x)
    # remove punctuation
    operator = str.maketrans('','',string.punctuation) #????
    x = x.translate(operator)
    if correct_spelling:
        x = correct_spellings(x)
    if remove_emojis:
        x = x.encode('ascii', 'ignore').decode('utf8').strip()
    if remove_stop_words:
        x = ' '.join([word for word in x.split(' ') if word not in STOPWORDS])
    return x

In [ ]:
data['cleaned_text'] = data['question_text'].apply(questions_cleaning)

In [ ]:
data.to_csv("/content/drive/My Drive/quora_train_clean.csv")

Writing the intermediate file to Google drive.  We can use load this file directly to test the algorithm

In [ ]:
data = pd.read_csv("/content/drive/My Drive/quora_train_clean.csv",encoding='iso-8859-1')

In [ ]:
data.shape

(65306, 6)